In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.summary2 import summary_col

from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
df = pd.read_pickle('C:/IP_DATA/INQ_DATA.pkl')
df = df.sort_index()
df = df.dropna()
#test
df = df.reset_index()
df['regressor'] = 1

states = df.region.unique()
states = [x for x in states if str(x) != 'nan']

In [4]:

#df['dummy'] = 0
#df['trend'] = 0

#for i in range(0,len(df)):
#    df['trend'][i] = i
    
#for i in range(0,len(df)):
#    if i < 370000:
#        df['dummy'][i] = 0
#    else:
#        df['dummy'][i] = 1

In [5]:
#df = df.pivot_table(values='regressor', index = 'date', columns = 'region')

In [6]:
#df = df.fillna(0)

In [3]:
#creat a series of other crimes
corruption = df.loc[df['offences'].str.contains( 'orrup') ==True]
embezzlement = df.loc[df['offences'].str.contains( 'ecula') ==True]
environmental = df.loc[df['offences'].str.contains( 'ambient') ==True]
desobedience = df.loc[df['offences'].str.contains( 'esobed') ==True]
extortive_corruption = df.loc[df['offences'].str.contains( 'oncuss') ==True]
swindle = df.loc[df['offences'].str.contains( 'stelionato') ==True]
theft= df.loc[df['offences'].str.contains( 'oubo|urto|o de domic') ==True]
drugs_related = df.loc[df['offences'].str.contains( 'rogas') ==True]
against_property = df.loc[df['offences'].str.contains( 'contra o Patri') ==True]
finance_related = df.loc[df['offences'].str.contains( 'Financeiro') ==True]
currency_falsification = df.loc[df['offences'].str.contains( 'Moeda Falsa') ==True]
fake_id = df.loc[df['offences'].str.contains( 'alsidade ide') ==True]
gun_restrictions = df.loc[df['offences'].str.contains( 'Armas') ==True]
authority_abuse = df.loc[df['offences'].str.contains( 'Abuso de Autoridade') ==True]
procurement_fraud = df.loc[df['offences'].str.contains( 'licita') ==True]
crimes_corruption = df.loc[df['offences'].str.contains('eculato|orrup|oncus' )==True]
other_crimes = df.loc[df['offences'].str.contains('eculato|orrup|oncus' )==False]


In [4]:
#df = df.set_index(['date','region'])

In [5]:
crimes_corruption = crimes_corruption.pivot_table(values='regressor', index = 'date', columns = 'region')
crimes_corruption = crimes_corruption.fillna(0)

In [6]:
crimes_corruption['crimes_corruption'] = crimes_corruption.sum(axis=1)

In [7]:
crimes_corruption = crimes_corruption.resample('D').sum()

#august 2013 is 1669
#january 2018 is 3284

In [63]:
crimes_corruption['dummy']= 0 
crimes_corruption['dummy2']= 0 
crimes_corruption['cons'] = 1
for i in range(1669,len(crimes_corruption)):
    crimes_corruption['dummy'][i] = 1
    
for i in range(3284,len(crimes_corruption)):
    crimes_corruption['dummy2'][i] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
crimes_corruption['enforce'] = crimes_corruption[['PR','DF','RJ','SP']].sum(axis=1)
crimes_corruption['enf'] = 0
crimes_corruption['int'] = 0

#for i in range(1669,len(crimes_corruption)):
#    crimes_corruption['int'][i] = crimes_corruption['enforce'] [i]
    
for i in range(0,len(crimes_corruption)):
    if crimes_corruption['enforce'][i]> 0:
        crimes_corruption['enf'][i] = 1
    else:
        pass


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [65]:
crimes_corruption.head()

region,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,...,SP,TO,crimes_corruption,dummy,cons,enforce,enf,int,enf2,dummy2
date,,,,,,,,,,,,,,,,,,,,,
2009-01-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,5.0,0,1,0.0,0,0,0,0
2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0.0,0,0,0,0
2009-01-06,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,6.0,0,1,2.0,1,0,0,0
2009-01-07,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,4.0,0,1,1.0,1,0,0,0
2009-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0.0,0,0,0,0


In [76]:
results1 = sm.OLS(crimes_corruption['crimes_corruption'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(crimes_corruption['crimes_corruption'], 
                  crimes_corruption[['cons','dummy','enf']]).fit()
results3 = sm.OLS(crimes_corruption['crimes_corruption'], 
                  crimes_corruption[['cons','dummy','dummy2','enf']]).fit()

stargazer = Stargazer([results1,results2,results3])
stargazer.covariate_order(['cons','dummy','dummy2','enf'])
HTML(stargazer.render_html())

In [77]:
results1 = sm.OLS(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy','enf']]).fit()
results3 = sm.OLS(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy','dummy2','enf']]).fit()
results4 = sm.OLS(crimes_corruption['SP'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
results5 = sm.OLS(crimes_corruption['SP'], crimes_corruption[['cons','dummy','enf','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
results6 = sm.OLS(crimes_corruption['SP'], crimes_corruption[['cons','dummy','dummy2','enf','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
stargazer = Stargazer([results1,results2,results3,results4,results5,results6])
stargazer.covariate_order(['cons','dummy','dummy2','enf','PR','DF','RJ'])
HTML(stargazer.render_html())

In [78]:
results1 = sm.Poisson(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.Poisson(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy','enf']]).fit()
results3 = sm.Poisson(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy','enf','dummy2']]).fit()
results4 = sm.Poisson(crimes_corruption['SP'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
results5 = sm.Poisson(crimes_corruption['SP'], crimes_corruption[['cons','dummy','enf','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
results6 = sm.Poisson(crimes_corruption['SP'], crimes_corruption[['cons','dummy','dummy2','enf2','RS','BA','PB','MS','RO','MT','PA',
                                                             'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','PR','DF','RJ']]).fit()
print(results1.summary())
print(results2.summary())
print(results3.summary())
print(results4.summary())
print(results5.summary())
print(results6.summary())

Optimization terminated successfully.
         Current function value: 0.731722
         Iterations 4
         Current function value: 0.542864
         Iterations: 35
         Current function value: 0.542287
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.653162
         Iterations 6


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


         Current function value: 0.536087
         Iterations: 35
         Current function value: 0.627187
         Iterations: 35
                          Poisson Regression Results                          
Dep. Variable:                     SP   No. Observations:                 4350
Model:                        Poisson   Df Residuals:                     4348
Method:                           MLE   Df Model:                            1
Date:                Sat, 29 Feb 2020   Pseudo R-squ.:                0.001643
Time:                        15:12:45   Log-Likelihood:                -3183.0
converged:                       True   LL-Null:                       -3188.2
                                        LLR p-value:                  0.001209
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
cons          -1.1155      0.043    -26.090      0.000      -1.199      -1.032

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                          Poisson Regression Results                          
Dep. Variable:                     SP   No. Observations:                 4350
Model:                        Poisson   Df Residuals:                     4321
Method:                           MLE   Df Model:                           28
Date:                Sat, 29 Feb 2020   Pseudo R-squ.:                  0.2686
Time:                        15:12:45   Log-Likelihood:                -2332.0
converged:                      False   LL-Null:                       -3188.2
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
cons         -25.5023   7742.323     -0.003      0.997   -1.52e+04    1.51e+04
dummy          0.1479      0.058      2.535      0.011       0.034       0.262
enf           24.9381   7742.323      0.003      0.9

In [87]:
results1 = sm.OLS(crimes_corruption['PR'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(crimes_corruption['PR'], 
                  crimes_corruption[['cons','dummy','enf']]).fit()
results3 = sm.OLS(crimes_corruption['PR'], 
                  crimes_corruption[['cons','dummy','enf','dummy2']]).fit()
results4 = sm.OLS(crimes_corruption['PR'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','DF','RJ']]).fit()
results5 = sm.OLS(crimes_corruption['PR'], crimes_corruption[['cons','dummy','enf','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','DF','RJ']]).fit()
results6 = sm.OLS(crimes_corruption['PR'], crimes_corruption[['cons','dummy','enf','dummy2','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','DF','RJ']]).fit()
stargazer = Stargazer([results1,results2,results3,results4,results5,results6])
stargazer.covariate_order(['cons','dummy','dummy2','enf','SP','DF','RJ'])
HTML(stargazer.render_latex())

In [84]:
results1 = sm.Poisson(crimes_corruption['PR'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.Poisson(crimes_corruption['PR'], 
                  crimes_corruption[['cons','dummy','dummy2']]).fit()
results3 = sm.Poisson(crimes_corruption['PR'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','DF','RJ']]).fit()
results4 = sm.Poisson(crimes_corruption['PR'], crimes_corruption[['cons','dummy','dummy2','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','DF','RJ']]).fit()
print(results1.summary())
print(results2.summary())
print(results3.summary())
print(results4.summary())
#print(results5.summary())
#print(results6.summary())

Optimization terminated successfully.
         Current function value: 0.535402
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.526684
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.511741
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.506320
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     PR   No. Observations:                 4350
Model:                        Poisson   Df Residuals:                     4348
Method:                           MLE   Df Model:                            1
Date:                Sat, 29 Feb 2020   Pseudo R-squ.:                 0.03323
Time:                        15:19:00   Log-Likelihood:                -2329.0
converged:                       True   LL-Null:                       -2409.0
                                       

In [88]:
results1 = sm.Poisson(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.Poisson(crimes_corruption['SP'], 
                  crimes_corruption[['cons','dummy','dummy2']]).fit()
results3 = sm.Poisson(crimes_corruption['SP'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','RJ','DF','PR']]).fit()
results4 = sm.Poisson(crimes_corruption['SP'], crimes_corruption[['cons','dummy','dummy2','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','RJ','DF','PR']]).fit()
print(results1.summary())
print(results2.summary())
print(results3.summary())
print(results4.summary())
#print(results5.summary())
#print(results6.summary())

Optimization terminated successfully.
         Current function value: 0.731722
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.723155
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.653162
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.650636
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     SP   No. Observations:                 4350
Model:                        Poisson   Df Residuals:                     4348
Method:                           MLE   Df Model:                            1
Date:                Sat, 29 Feb 2020   Pseudo R-squ.:                0.001643
Time:                        15:40:40   Log-Likelihood:                -3183.0
converged:                       True   LL-Null:                       -3188.2
                                       

In [86]:
results1 = sm.Poisson(crimes_corruption['DF'], 
                  crimes_corruption[['cons','dummy']]).fit()
results2 = sm.Poisson(crimes_corruption['DF'], 
                  crimes_corruption[['cons','dummy','dummy2']]).fit()
results3 = sm.Poisson(crimes_corruption['DF'], crimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','PR','RJ']]).fit()
results4 = sm.Poisson(crimes_corruption['DF'], crimes_corruption[['cons','dummy','dummy2','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','SP','PR','RJ']]).fit()
print(results1.summary())
print(results2.summary())
print(results3.summary())
print(results4.summary())
#print(results5.summary())
#print(results6.summary())

Optimization terminated successfully.
         Current function value: 0.709213
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.706117
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620337
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.620326
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     DF   No. Observations:                 4350
Model:                        Poisson   Df Residuals:                     4348
Method:                           MLE   Df Model:                            1
Date:                Sat, 29 Feb 2020   Pseudo R-squ.:                 0.01096
Time:                        15:21:13   Log-Likelihood:                -3085.1
converged:                       True   LL-Null:                       -3119.3
                                       

In [15]:
#do not collapse for now - > just to show how different they are

mcrimes_corruption = crimes_corruption.resample('M').sum()
mcrimes_corruption['cons'] = 1
mcrimes_corruption['dummy'] = 0
for i in range(69,len(mcrimes_corruption)):
    mcrimes_corruption['dummy'][i] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [16]:
results1 = sm.OLS(mcrimes_corruption['PR'], mcrimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(mcrimes_corruption['PR'], mcrimes_corruption[['cons','dummy','SP','DF','RJ','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
results3 = sm.OLS(mcrimes_corruption['PR'], mcrimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
stargazer = Stargazer([results1,results2,results3])
stargazer.covariate_order(['cons','dummy'])
HTML(stargazer.render_html())


In [17]:
results1 = sm.OLS(mcrimes_corruption['DF'], mcrimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(mcrimes_corruption['DF'], mcrimes_corruption[['cons','dummy','SP','PR','RJ','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
results3 = sm.OLS(mcrimes_corruption['DF'], mcrimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
stargazer = Stargazer([results1,results2,results3])
stargazer.covariate_order(['cons','dummy'])
HTML(stargazer.render_html())


In [18]:
results1 = sm.OLS(mcrimes_corruption['RJ'], mcrimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(mcrimes_corruption['RJ'], mcrimes_corruption[['cons','dummy','SP','PR','DF','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
results3 = sm.OLS(mcrimes_corruption['RJ'], mcrimes_corruption[['cons','dummy','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES']]).fit()
stargazer = Stargazer([results1,results2,results3])
stargazer.covariate_order(['cons','dummy'])
HTML(stargazer.render_html())

In [19]:
results1 = sm.OLS(mcrimes_corruption['crimes_corruption'], mcrimes_corruption[['cons','dummy']]).fit()
results2 = sm.OLS(mcrimes_corruption['crimes_corruption'], mcrimes_corruption[['cons','dummy','SP','PR','DF','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','RJ']]).fit()
results3 = sm.OLS(mcrimes_corruption['crimes_corruption'], mcrimes_corruption[['cons','dummy','int','SP','PR','DF','RS','BA','PB','MS','RO','MT','PA',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES','RJ']]).fit()
stargazer = Stargazer([results1,results2,results3])
stargazer.covariate_order(['cons','dummy','int','RS','BA','PB','MS','RO','MT','PA','RJ',
                                                              'AC','AM','MG','PI','AL','MA','GO','TO','CE','AP','RR',
                                                              'RN','PE','SE','SC','ES'])
HTML(stargazer.render_html())

In [31]:
x = crimes_corruption.resample('Y').sum()

In [32]:
x = x.transpose()

In [35]:
HTML(x.to_latex())